# water-hBN-structure-generator

In [2]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

### make water-hBN structure

In [47]:
dis = 11.37 # 20vmd
spc = 2.0
L = [[0.0, 15.0], [0.0, 13.0], [0.0, dis]]

wat = np.loadtxt("bulkw.xyz", skiprows=2, usecols=(1,2,3))
print ("number of water:", int(len(wat)/3))

hbn = np.loadtxt("hbn.xyz", skiprows=2, usecols=(1,2,3))
print ("number of atoms in mono-layer hBN:", len(hbn))
    
sub = []
for i in range(len(hbn)):
    sub.append( hbn[i,:] )
#     sub.append( hbn[i,:] + np.array([0,0,dis]) )
nsub = len(sub)
    
sol = []
for i in range(0, len(wat/3), 3):
    if (wat[i,0] > L[0][0]+spc/2) and (wat[i,0] < L[0][1]-spc/2):
        if (wat[i,1] > L[1][0]+spc/2) and (wat[i,1] < L[1][1]-spc/2):
            if (wat[i,2] > L[2][0]+spc) and (wat[i,2] < L[2][1]-spc):
                for j in range(3):
                    sol.append( wat[i+j,:] )
nsol = len(sol)
print ("number of selected water:", int(nsol/3))

print ("total number of atoms:", nsub+nsol)

number of water: 358
number of atoms in mono-layer hBN: 72
number of selected water: 31
total number of atoms: 165


### save to .xyz

In [48]:
# only water
ofile = open("selw.xyz", 'w')
ofile.write("%d\n"%(nsol))
ofile.write("\n")
# solution
for i in range(0, nsol, 3):
    for j in range(3):
        ofile.write("%s  %10.5f %10.5f %10.5f\n"%(['O', 'H', 'H'][j], sol[i+j][0], sol[i+j][1], sol[i+j][2]))
ofile.close()

# water-substrate
ofile = open("w-hbn.xyz", 'w')
ofile.write("%d\n"%(nsub+nsol))
ofile.write("\n")
# substrate
for i in range(0, nsub):
        ofile.write("%s  %10.5f %10.5f %10.5f\n"%(['B', 'N'][i%2], sub[i][0], sub[i][1], sub[i][2]))
# solution
for i in range(0, nsol, 3):
    for j in range(3):
        ofile.write("%s  %10.5f %10.5f %10.5f\n"%(['O', 'H', 'H'][j], sol[i+j][0], sol[i+j][1], sol[i+j][2]))
ofile.close()

### save to .lmp

In [49]:
ofile = open("w-hbn.lmp", 'w')
ofile.write("\n%d atoms\n4 atom types"%(nsub+nsol))
ofile.write("\n%d bonds\n1 bond types"%(nsol/3*2))
ofile.write("\n%d angles\n1 angle types"%(nsol/3))
ofile.write("\n%f %f xlo xhi\n"%(L[0][0], L[0][1]))
ofile.write("%f %f ylo yhi\n"%(L[1][0], L[1][1]))
ofile.write("%f %f zlo zhi\n"%(L[2][0], L[2][1]))
ofile.write("\nAtoms\n\n")
aid = 0; mid = 1
# substrate
for i in range(0, nsub):
        aid += 1
        ofile.write("%3d %2d %2s  0.00 %10.5f %10.5f %10.5f\n"%(aid, mid, ['2', '3'][i%2], sub[i][0], sub[i][1], sub[i][2]))
# solution
for i in range(0, nsol, 3):
    mid +=1
    for j in range(3):
        aid += 1
        ofile.write("%3d %2d %2s  0.00 %10.5f %10.5f %10.5f\n"%(aid, mid, ['4', '1', '1'][j], sol[i+j][0], sol[i+j][1], sol[i+j][2]))
ofile.close()

# add bonds and angles
import os
os.system("python atl.xyz-to-tip3p.py selw.xyz O H %d"%nsub)
os.system("cat w-hbn.lmp tip3p.dat > w-hbn.lmp.full")


0